In [13]:
from Model_Training import *
import warnings
from Bus import Bus
from SimulationEngine import SimulationEngine
from datetime import datetime

warnings.filterwarnings('ignore')

In [14]:
buses_model = Buses_model()

days = 50

In [3]:
buses_model.generate_model()

Iniciando modelado de la distribución de Tiempos de Viaje...
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_0700-0930 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_1330-1800 entrenado.
  -> Modelo de Tiempo A2-GLAXO_0600-0700 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 9. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 11. Using x_max as inverse.
Error: No valid symbolic solutio

In [16]:
TIME_INTERVALS = [
    ('06:00', '07:00'),
    ('07:00', '09:30'),
    ('09:30', '13:30'),
    ('13:30', '18:00'),
    ('18:30', '22:15')
]

# 2. Helper function to define Fleet Configuration per interval
def get_fleet_config(start_t, end_t):
    """
    Returns the list of Bus objects configured specifically for the given interval.
    You can manually change the number of buses or their parameters here.
    """
    
    # Common parameters can be defined here if needed, or inside the ifs
    
    if start_t == '06:00':
        # Early morning: Maybe fewer buses or specific start locations
        eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, 
                     start_time=start_t, end_time=end_t, r_base=4.5, fp=0.0052, c_hour=3, samples=80, route_type=1)
        eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, 
                     start_time=start_t, end_time=end_t, r_base=12.5, fp=0.0281, c_hour=1, samples=80, route_type=2)
        return [eco_07, eco_50]

    elif start_t == '07:00':
        # Rush Hour: Example of adding a 3rd bus manually
        eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, 
                     start_time=start_t, end_time=end_t, r_base=4.5, fp=0.0052, c_hour=3, samples=80, route_type=1)
        eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, 
                     start_time=start_t, end_time=end_t, r_base=12.5, fp=0.0281, c_hour=1, samples=80, route_type=2)
        
        return [eco_07, eco_50]

    elif start_t == '09:30':
        # Mid-day
        eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, 
                     start_time=start_t, end_time=end_t, r_base=4.5, fp=0.0052, c_hour=3, samples=80, route_type=1)
        eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, 
                     start_time=start_t, end_time=end_t, r_base=12.5, fp=0.0281, c_hour=1, samples=80, route_type=2)
        return [eco_07, eco_50]
        
    elif start_t == '13:30':
        eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, 
                     start_time=start_t, end_time=end_t, r_base=4.5, fp=0.0052, c_hour=3, samples=80, route_type=1)
        eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, 
                     start_time=start_t, end_time=end_t, r_base=12.5, fp=0.0281, c_hour=1, samples=80, route_type=2)
        return [eco_07, eco_50]
    else:
        # Default config for other intervals (18:30, etc.)
        eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, 
                     start_time=start_t, end_time=end_t, r_base=4.5, fp=0.0052, c_hour=3, samples=80, route_type=1)
        eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, 
                     start_time=start_t, end_time=end_t, r_base=12.5, fp=0.0281, c_hour=1, samples=80, route_type=2)
        return [eco_07, eco_50]

In [ ]:
daily_stop_stats = []
daily_bus_stats = []

for day in range(days):
    
    # Loop through each time interval within the day
    for start_time, end_time in TIME_INTERVALS:
        
        # 1. Get the fleet configuration for this specific interval
        total_buses = get_fleet_config(start_time, end_time)
        N_BUSES = len(total_buses)

        # 2. Re-Initialize Model
        pax_model = Pax_model()
        pax_model.generate_model(n_buses=N_BUSES)

        # 3. Run Sim
        # Note: We pass the specific start/end time from the loop
        sim = SimulationEngine(total_buses, pax_model, (start_time, end_time), show_process=False)
        sim.setup()
        sim.run()
        
        # 4. Get Metrics
        day_stop_stats, day_bus_stats = sim.get_metrics(show_metrics=False)
        
        # 5. Tag the Data
        # We add the Day AND the Interval Label so we can group by them later
        interval_label = f"{start_time}-{end_time}"
        
        day_stop_stats['day'] = day
        day_stop_stats['interval'] = interval_label
        
        day_bus_stats['day'] = day
        day_bus_stats['interval'] = interval_label
        
        daily_stop_stats.append(day_stop_stats)
        daily_bus_stats.append(day_bus_stats)

    # Optional: Print progress per day
    # print(f'Ended simulation of day {day+1}')

# --- AGGREGATION ---

# 1. Stop Stats
all_stops_df = pd.concat(daily_stop_stats, ignore_index=True)

# Group by Interval AND Origin to see how stops perform at different times of day
avg_stop_stats = all_stops_df.groupby(['interval', 'origin']).agg({
    'Avg_Wait_Min': 'mean',
    'Max_Wait_Min': 'max'
}).reset_index()

# 2. Bus Stats
all_buses_df = pd.concat(daily_bus_stats, ignore_index=True)

# Ensure types are correct
cols_to_float = ['Avg Occupancy', 'Total Cost', 'Trips Made', 'Total gas (L)']
for col in cols_to_float:
    all_buses_df[col] = all_buses_df[col].astype(float)

# Group by Interval AND Bus ID
avg_bus_stats = all_buses_df.groupby(['interval', 'Bus ID']).agg({
    'Avg Occupancy': 'mean',
    'Trips Made': 'mean', 
    'Total Cost': 'mean',
    'Total gas (L)': 'mean'
}).reset_index()

# --- OUTPUT CHECK ---
#print("--- Stop Stats per Interval ---")
#print(avg_stop_stats.head(10))
#print("\n--- Bus Stats per Interval ---")
#print(avg_bus_stats.head(10))

In [11]:
avg_bus_stats

,interval,Bus ID,Avg Occupancy,Trips Made,Total Cost,Total gas (L)
0,06:00-07:00,07,6.94,4.8,204.48,4.052
1,06:00-07:00,50,14.24,4.8,119.04,1.254
2,07:00-09:30,07,14.40,11.6,494.16,10.422
3,07:00-09:30,50,24.26,13.0,322.40,3.568
4,09:30-13:30,07,12.22,22.2,945.72,17.684
5,09:30-13:30,50,25.60,21.4,530.72,5.838
6,13:30-18:00,07,16.10,23.2,988.32,19.664
7,13:30-18:00,50,31.96,24.0,595.20,6.552
8,18:30-22:15,07,5.18,26.6,1133.16,18.084
9,18:30-22:15,50,9.32,26.6,659.68,5.918


In [7]:
old_bus_stats = pd.read_csv('bus_stats.csv')
old_bus_stats

,interval,Bus ID,Avg Occupancy,Trips Made,Total Cost,Total gas (L)
0,06:00-07:00,7,4.24,5.4,230.04,3.948
1,06:00-07:00,50,16.62,5.0,124.00,1.208
2,07:00-09:30,7,9.90,11.6,494.16,9.846
3,07:00-09:30,50,28.18,12.6,312.48,3.478
4,07:00-09:30,99,20.52,12.0,360.00,5.764
5,09:30-13:30,7,10.72,19.8,843.48,16.268
6,09:30-13:30,50,37.08,21.0,520.80,5.792
7,13:30-18:00,7,16.04,24.8,1056.48,19.050
8,13:30-18:00,50,34.92,23.4,580.32,6.614
9,18:30-22:15,7,3.96,26.4,1124.64,17.038


In [ ]:
avg_stop_stats

In [9]:
old_stop_stats = pd.read_csv('stop_stats.csv')

old_stop_stats

,interval,origin,Avg_Wait_Min,Max_Wait_Min
0,06:00-07:00,A2,8.238000,17.600000
1,06:00-07:00,DISNEY,8.674000,21.290000
2,06:00-07:00,GLAXO,5.220000,16.610000
3,06:00-07:00,General,8.402437,21.293848
4,07:00-09:30,A2,7.252000,26.620000
5,07:00-09:30,DISNEY,7.984000,28.250000
6,07:00-09:30,GLAXO,7.326000,19.620000
7,07:00-09:30,General,7.772166,28.254898
8,09:30-13:30,A2,10.618000,36.870000
9,09:30-13:30,DISNEY,11.010000,30.670000
